In [2]:
from PIL import Image, ImageDraw, ImageFont
import json
import os
import math
from typing import Dict, List, Tuple, Any


In [ ]:
#Unicode script detectors 
def has_devanagari(text: str) -> bool:
    return any('\u0900' <= ch <= '\u097F' for ch in text)

def has_bengali(text: str) -> bool:
    return any('\u0980' <= ch <= '\u09FF' for ch in text)

#Load font with fallback 
def load_font_preferred(font_paths: Dict[str, str], size: int) -> ImageFont.FreeTypeFont:
    """
    font_paths is a dict e.g. {"devanagari": "fonts/NotoSansDevanagari-Regular.ttf",
                               "bengali": "fonts/NotoSansBengali-Regular.ttf",
                               "latin": "fonts/NotoSans-Regular.ttf"}
    We'll return a best available font from 'latin' fallback if preferred missing.
    """
    try:
        return ImageFont.truetype(font_paths, size)
    except Exception:
        try:
            return ImageFont.load_default()
        except Exception:
            raise RuntimeError("No available font; please install Pillow default fonts or set font paths.")

#Convert percentage bbox to pixel bbox
def percent_to_pixels(bbox_pct: Tuple[float, float, float, float], image_size: Tuple[int, int]) -> Tuple[int, int, int, int]:
    """
    bbox_pct: (x_percent, y_percent, width_percent, height_percent) with values like 21.2074
    Returns pixel bbox (left, top, width_px, height_px)
    """
    W, H = image_size
    x_pct, y_pct, w_pct, h_pct = bbox_pct
    left = int((x_pct / 100.0) * W)
    top  = int((y_pct / 100.0) * H)
    w_px = int((w_pct / 100.0) * W)
    h_px = int((h_pct / 100.0) * H)
    return left, top, w_px, h_px

# Measure text size given a font
def text_size_px(text: str, font: ImageFont.FreeTypeFont) -> Tuple[int, int]:
    try:
        return font.getsize(text)
    except Exception:
        return (len(text) * int(font.size * 0.6), font.size)


In [ ]:
# Demo rules for two states
RULES = {
    "Uttar Pradesh": {
        "V_STATE_RESTRICTION_EN": "FOR SALE IN U.P. ONLY",
        "V_STATE_RESTRICTION_REG": "उप में केवल बिक्री के लिए",
        "V_AGE_RESTR_EN": "Not for sale to persons below 25 years of age.",
        "NOT_SOLD_IN": "Not for sale in Bihar",
        "V_HEALTH_WARN_EN": "CONSUMPTION OF ALCOHOL IS INJURIOUS TO HEALTH, BE SAFE-DON'T DRINK AND DRIVE.",
        "V_HEALTH_WARN_REG": "शराब स्वास्थ्य के लिए हानिकारक है.",
        "IMPORT_LISC": "1001245678",
        "V_StateLicense": "UP. EXCISE REGD. NO. E AC-45/110W-106046"
    },
    "Rajasthan": {
        "V_STATE_RESTRICTION_EN": "FOR SALE IN Rajasthan ONLY",
        "V_STATE_RESTRICTION_REG": "उप में केवल बिक्री के लिए",
        "V_AGE_RESTR_EN": "Not for sale to persons below 21 years of age.",
        "NOT_SOLD_IN": "Not for sale in Madhya Pradesh",
        "V_HEALTH_WARN_EN": "CONSUMPTION OF ALCOHOL IS INJURIOUS TO HEALTH, BE SAFE-DON'T DRINK AND DRIVE.",
        "V_HEALTH_WARN_REG": "शराब स्वास्थ्य के लिए हानिकारक है.",
        "IMPORT_LISC": "1001234567",
        "V_StateLicense": "RA. EXCISE REGD. NO. E AC-45/110W-10000"
    }
}


In [5]:
def load_template_json(path: str) -> List[Dict[str, Any]]:
    """
    Expects the cleaned JSON structure:
    [
      {
        "image": "3d44cffd-unnamed.png",
        "image_width": 341,
        "image_height": 512,
        "regions": [
          {"label": "V_STATE_RESTRICTION_EN", "x": 21.2074, "y": 54.2298, "width": 44.1177, "height": 2.4744},
          ...
        ]
      },
      ...
    ]
    """
    with open(path, "/Users/ayushtiwari/Desktop/label/template_clean.json.json", encoding="utf-8") as f:
        data = json.load(f)
    return data

# Small helper to find template entry for an image filename
def find_template_for_image(template_list: List[Dict[str, Any]], image_basename: str) -> Dict[str, Any]:
    for entry in template_list:
        # compare just basenames (safe)
        if os.path.basename(entry.get("image", "")) == os.path.basename(image_basename):
            return entry
    raise ValueError(f"No template found for image {image_basename}")


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os

# Using Default fonts
def get_default_font(size):
    try:
        return ImageFont.truetype("arial.ttf", size)
    except:
        try:
            return ImageFont.truetype("DejaVuSans.ttf", size)
        except:
            return ImageFont.load_default()


def fit_font_size_to_box(draw, text, box_w, box_h, max_font=80, min_font=8, padding=2):
    """
    Dynamically shrink font until text fits inside region box
    """
    for size in range(max_font, min_font, -1):
        font = get_default_font(size)

        # NEW Pillow method
        bbox = draw.textbbox((0,0), text, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]

        if w <= (box_w - padding) and h <= (box_h - padding):
            return font

    return get_default_font(min_font)


# Rendering
def render_label(master_img_path,
                 template_entry,
                 state_name,
                 rules,
                 output_path,
                 debug=True):

    img = Image.open(master_img_path).convert("RGBA")
    W, H = img.size
    draw = ImageDraw.Draw(img)

    regions = template_entry.get("regions", [])
    state_rules = rules.get(state_name)

    if state_rules is None:
        raise ValueError(f"No rules found for state {state_name}")

    for region in regions:
        label = region.get("label")

        if label not in state_rules:
            continue

        text_for_label = state_rules[label]

        # % to pixels
        left = int((region["x"]/100)*W)
        top = int((region["y"]/100)*H)
        w_px = int((region["width"]/100)*W)
        h_px = int((region["height"]/100)*H)

        w_px = max(1, w_px)
        h_px = max(1, h_px)

        # Fit font
        font = fit_font_size_to_box(draw, text_for_label, w_px, h_px)

        bbox = draw.textbbox((0,0), text_for_label, font=font)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]

        # center text
        x_text = left + (w_px - text_w)/2
        y_text = top + (h_px - text_h)/2

        # white background for readability
        label_upper = label.upper()
        if any(k in label_upper for k in ("HEALTH","STATE","AGE","NOT","WARN")):
            overlay = Image.new('RGBA', img.size, (255,255,255,0))
            od = ImageDraw.Draw(overlay)
            pad = 2
            od.rectangle([left-pad, top-pad, left+w_px+pad, top+h_px+pad],
                         fill=(255,255,255,220))
            img = Image.alpha_composite(img, overlay)
            draw = ImageDraw.Draw(img)

        # draw text
        draw.text((x_text, y_text), text_for_label, fill=(0,0,0,255), font=font)

        # debug boxes
        if debug:
            draw.rectangle([left, top, left+w_px, top+h_px], outline="red", width=1)
            draw.text((left, top-12), label, fill="red", font=get_default_font(10))

    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
    img.convert("RGB").save(output_path, "PNG")

    print("✅ Final label saved to:", output_path)
    return output_path


In [17]:
MASTER_IMAGE_PATH = r"/Users/ayushtiwari/Desktop/label/Master Label Sample.png"
TEMPLATE_JSON_PATH = r"/Users/ayushtiwari/Desktop/label/template_clean.json"
OUTPUT_PATH = r"/Users/ayushtiwari/Desktop/label/output/final_label.png"

STATE = "Uttar Pradesh"   # or Rajasthan

import json
with open(TEMPLATE_JSON_PATH,"r",encoding="utf-8") as f:
    templates = json.load(f)

template_entry = templates[1]   # use first template for demo

render_label(
    MASTER_IMAGE_PATH,
    template_entry,
    STATE,
    RULES,
    OUTPUT_PATH,
    debug=True
)


AttributeError: 'ImageDraw' object has no attribute 'textsize'

In [27]:
from PIL import Image, ImageDraw, ImageFont
import json, os, sys
from typing import List, Dict, Tuple, Any

# Utility functions

def get_default_font(size: int) -> ImageFont.FreeTypeFont:
    """Return a usable system font. No external download required."""
    for name in ("arial.ttf", "DejaVuSans.ttf"):
        try:
            return ImageFont.truetype(name, size)
        except Exception:
            continue
    return ImageFont.load_default()

def percent_to_pixels(bbox_pct: Tuple[float,float,float,float], image_size: Tuple[int,int]) -> Tuple[int,int,int,int]:
    """Convert [x%,y%,w%,h%] in 0..100 -> pixel box (left,top,width,height)."""
    W, H = image_size
    x_pct, y_pct, w_pct, h_pct = bbox_pct
    left = int((x_pct/100.0) * W)
    top  = int((y_pct/100.0) * H)
    w_px = int((w_pct/100.0) * W)
    h_px = int((h_pct/100.0) * H)
    return left, top, w_px, h_px

def fit_font_size_to_box(draw: ImageDraw.ImageDraw, text: str, box_w: int, box_h: int,
                         max_font: int = 80, min_font: int = 6, padding: int = 4) -> ImageFont.FreeTypeFont:
    """
    Find largest font size (from max_font downwards) that fits text inside box_w x box_h.
    Uses draw.textbbox (Pillow modern API).
    """
    for size in range(max_font, min_font - 1, -1):
        font = get_default_font(size)
        bbox = draw.textbbox((0,0), text, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        if w + padding*2 <= box_w and h + padding*2 <= box_h:
            return font
    return get_default_font(min_font)


# Template loading helpers
def load_template_json(path: str) -> List[Dict[str,Any]]:
    """Load cleaned template JSON"""
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Template JSON not found at: {path}")
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if not isinstance(data, list) or len(data) == 0:
        raise ValueError("Template JSON must be a non-empty list of template entries.")
    return data

def find_template_for_image_or_first(template_list: List[Dict[str,Any]], image_path: str) -> Dict[str,Any]:
    """Try filename matching (basename). If not found, return first template with a WARNING."""
    base = os.path.basename(image_path)
    for entry in template_list:
        if os.path.basename(entry.get("image","")) == base:
            return entry
    # fallback
    print(f"⚠️ Warning: no matching template filename for '{base}'. Falling back to first template entry ('{template_list[0].get('image')}').")
    return template_list[0]

# Renderer
def render_label(master_img_path: str,
                 template_entry: Dict[str,Any],
                 state_name: str,
                 rules: Dict[str,Dict[str,str]],
                 output_path: str,
                 debug: bool = True) -> str:
    """Render the master image with state-specific variable texts and save to output_path."""
    if not os.path.isfile(master_img_path):
        raise FileNotFoundError(f"Master image not found: {master_img_path}")

    img = Image.open(master_img_path).convert("RGBA")
    W, H = img.size
    draw = ImageDraw.Draw(img)

    regions = template_entry.get("regions", [])
    if not isinstance(regions, list):
        raise ValueError("Template entry missing 'regions' list.")

    state_rules = rules.get(state_name)
    if state_rules is None:
        raise ValueError(f"No rules found for state '{state_name}'")

    for region in regions:
        label = region.get("label")
        if label is None:
            continue

        # Map label -> text (skip if rule missing)
        if label not in state_rules:
            if debug:
                print(f" - skipping region '{label}' (no rule for this state)")
            continue
        text_for_label = state_rules[label]

        # convert percentage -> pixels
        left, top, w_px, h_px = percent_to_pixels(
            (region.get("x",0), region.get("y",0), region.get("width",0), region.get("height",0)), (W,H)
        )
        w_px = max(1, w_px); h_px = max(1, h_px)

        # Fit font
        font = fit_font_size_to_box(draw, text_for_label, w_px, h_px)

        # Measure final text bbox
        tb = draw.textbbox((0,0), text_for_label, font=font)
        text_w = tb[2] - tb[0]
        text_h = tb[3] - tb[1]

        # Center text (single-line assumption)
        x_text = left + (w_px - text_w) / 2
        y_text = top + (h_px - text_h) / 2

        # Optional semi-opaque white bg for legibility (for statutory text)
        label_upper = (label or "").upper()
        if any(k in label_upper for k in ("HEALTH","STATE","AGE","NOT","WARN","RESTRICTION")):
            overlay = Image.new('RGBA', img.size, (255,255,255,0))
            od = ImageDraw.Draw(overlay)
            pad = 2
            od.rectangle([left-pad, top-pad, left+w_px+pad, top+h_px+pad], fill=(255,255,255,220))
            img = Image.alpha_composite(img, overlay)
            draw = ImageDraw.Draw(img)  # refresh draw after composite

        # Draw text
        draw.text((x_text, y_text), text_for_label, fill=(0,0,0,255), font=font)

        # Debug visuals
        if debug:
            draw.rectangle([left, top, left+w_px, top+h_px], outline=(255,0,0,200), width=1)
            # small label tag
            tag_font = get_default_font(10)
            draw.text((left+2, top-12), label, font=tag_font, fill=(255,0,0,200))

    # Save output (ensure output folder exists and a filename is provided)
    out_dir = os.path.dirname(output_path) or "."
    os.makedirs(out_dir, exist_ok=True)
    # Convert to RGB for compatibility
    img.convert("RGB").save(output_path, format="PNG")

    print("Final label saved to:", output_path)
    return output_path



MASTER_IMAGE_PATH = r"/Users/ayushtiwari/Desktop/label/Master Label Sample.png"
TEMPLATE_JSON_PATH = r"/Users/ayushtiwari/Desktop/label/template_clean.json"
OUTPUT_PATH = r"/Users/ayushtiwari/Desktop/label/output/final_label.png"
STATE = "Uttar Pradesh"   


# Run
print("Loading template JSON from:", TEMPLATE_JSON_PATH)
templates = load_template_json(TEMPLATE_JSON_PATH)

template_entry = find_template_for_image_or_first(templates, MASTER_IMAGE_PATH)
print("Template chosen (template.image):", template_entry.get("image"))

print("Rendering for state:", STATE)
render_label(MASTER_IMAGE_PATH, template_entry, STATE, RULES, OUTPUT_PATH, debug=True)


Loading template JSON from: /Users/ayushtiwari/Desktop/label/template_clean.json
Template chosen (template.image): Master Label Sample.png
Rendering for state: Uttar Pradesh
Final label saved to: /Users/ayushtiwari/Desktop/label/output/final_label.png


'/Users/ayushtiwari/Desktop/label/output/final_label.png'

## Version 2
1. Font size fixed
2. White background fix
3. Position fix
4. Reg Language Fix

In [28]:
from PIL import Image, ImageDraw, ImageFont
import os, platform


def get_system_font(size):
    """
    Automatically choose best system font for Unicode (Hindi etc)
    No manual download required.
    """
    system = platform.system()

    font_candidates = []

    if system == "Darwin": 
        font_candidates = [
            "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
            "/System/Library/Fonts/Supplemental/Arial Unicode MS.ttf",
            "/System/Library/Fonts/Supplemental/Devanagari Sangam MN.ttf",
            "/Library/Fonts/Arial Unicode.ttf"
        ]

    elif system == "Windows":
        font_candidates = [
            "C:/Windows/Fonts/arialuni.ttf",
            "C:/Windows/Fonts/segoeui.ttf"
        ]

    else:  # Linux
        font_candidates = [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
        ]

    for path in font_candidates:
        if os.path.exists(path):
            return ImageFont.truetype(path, size)

    return ImageFont.load_default()

# AUTO FONT FIT TO BOX

def fit_font(draw, text, box_w, box_h):
    for size in range(80, 6, -1):
        font = get_system_font(size)
        bbox = draw.textbbox((0,0), text, font=font)
        w = bbox[2]-bbox[0]
        h = bbox[3]-bbox[1]

        if w <= box_w-4 and h <= box_h-4:
            return font

    return get_system_font(8)


# RENDER FUNCTION V2

def render_label(master_img_path,
                 template_entry,
                 state_name,
                 rules,
                 output_path,
                 debug=True):

    img = Image.open(master_img_path).convert("RGBA")
    W,H = img.size
    draw = ImageDraw.Draw(img)

    regions = template_entry["regions"]
    state_rules = rules[state_name]

    for region in regions:
        label = region["label"]

        if label not in state_rules:
            continue

        text = state_rules[label]

        # % → pixels
        left = int((region["x"]/100)*W)
        top = int((region["y"]/100)*H)
        w = int((region["width"]/100)*W)
        h = int((region["height"]/100)*H)

        w = max(1,w)
        h = max(1,h)

        font = fit_font(draw, text, w, h)

        bbox = draw.textbbox((0,0), text, font=font)
        text_w = bbox[2]-bbox[0]
        text_h = bbox[3]-bbox[1]

        #LEFT ALIGN
        x_text = left + 3
        y_text = top + (h - text_h)/2

        draw.text((x_text, y_text), text, fill=(0,0,0,255), font=font)

        # debug red box
        if debug:
            draw.rectangle([left,top,left+w,top+h], outline="red", width=1)

    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
    img.convert("RGB").save(output_path)

    print("FINAL LABEL GENERATED:", output_path)
    return output_path


In [29]:
MASTER_IMAGE_PATH = r"/Users/ayushtiwari/Desktop/label/Master Label Sample.png"
TEMPLATE_JSON_PATH = r"/Users/ayushtiwari/Desktop/label/template_clean.json"
OUTPUT_PATH = r"/Users/ayushtiwari/Desktop/label/output/final_label_v2.png"

STATE = "Uttar Pradesh"

import json
with open(TEMPLATE_JSON_PATH,"r",encoding="utf-8") as f:
    templates = json.load(f)

template_entry = templates[1]   # choose correct template

render_label(
    MASTER_IMAGE_PATH,
    template_entry,
    STATE,
    RULES,
    OUTPUT_PATH,
    debug=True
)


FINAL LABEL GENERATED: /Users/ayushtiwari/Desktop/label/output/final_label_v2.png


'/Users/ayushtiwari/Desktop/label/output/final_label_v2.png'

# Version 3
1. Fixed alignment
2. Fixed word spill

In [36]:
from PIL import Image, ImageDraw, ImageFont
import os, platform

# ==========================================
# SYSTEM FONT AUTO DETECTION
# ==========================================
def get_system_font(size):
    system = platform.system()

    if system == "Darwin":  # Mac
        fonts = [
            "/System/Library/Fonts/Supplemental/Arial Unicode MS.ttf",
            "/System/Library/Fonts/Supplemental/Devanagari Sangam MN.ttf"
        ]
    elif system == "Windows":
        fonts = ["C:/Windows/Fonts/arialuni.ttf","C:/Windows/Fonts/segoeui.ttf"]
    else:
        fonts = ["/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"]

    for f in fonts:
        if os.path.exists(f):
            return ImageFont.truetype(f, size)

    return ImageFont.load_default()

# ==========================================
# FONT FIT THAT FILLS BOX HEIGHT MAX
# ==========================================
def get_best_font(draw, text, box_w, box_h):
    """
    Find largest font that FILLS box height
    """
    best_font = None

    for size in range(200, 6, -1):   # start big
        font = get_system_font(size)

        bbox = draw.textbbox((0,0), text, font=font)
        text_w = bbox[2]-bbox[0]
        text_h = bbox[3]-bbox[1]

        # must fit inside
        if text_w <= box_w and text_h <= box_h:
            best_font = font
            break

    if best_font is None:
        best_font = get_system_font(8)

    return best_font

# ==========================================
# RENDER FUNCTION FINAL
# ==========================================
def render_label(master_img_path,
                 template_entry,
                 state_name,
                 rules,
                 output_path,
                 debug=True):

    img = Image.open(master_img_path).convert("RGBA")
    W,H = img.size
    draw = ImageDraw.Draw(img)

    regions = template_entry["regions"]
    state_rules = rules[state_name]

    for region in regions:
        label = region["label"]

        if label not in state_rules:
            continue

        text = state_rules[label]

        # % → pixel conversion
        left = int((region["x"]/100)*W)
        top = int((region["y"]/100)*H)
        w = int((region["width"]/100)*W)
        h = int((region["height"]/100)*H)

        w = max(1,w)
        h = max(1,h)

        # 🔥 best font that fills box
        font = get_best_font(draw, text, w, h)

        bbox = draw.textbbox((0,0), text, font=font)
        text_w = bbox[2]-bbox[0]
        text_h = bbox[3]-bbox[1]

        # ----------------------------------
        # LEFT aligned horizontally
        x_text = left + 3

        # TRUE vertical center
        y_text = top + (h - text_h)/2 - bbox[1]
        # ----------------------------------

        draw.text((x_text, y_text), text, fill=(0,0,0,255), font=font)

        if debug:
            draw.rectangle([left,top,left+w,top+h], outline="red", width=1)

    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
    img.convert("RGB").save(output_path)

    print("FINAL LABEL GENERATED:", output_path)
    return output_path


In [37]:
render_label(
    MASTER_IMAGE_PATH,
    template_entry,
    STATE,
    RULES,
    OUTPUT_PATH,
    debug=True
)


FINAL LABEL GENERATED: /Users/ayushtiwari/Desktop/label/output/final_label_v3.png


'/Users/ayushtiwari/Desktop/label/output/final_label_v3.png'

# Version 3 JW UP

In [59]:
# FINAL - Robust rendering cell (copy-paste entire cell)
from PIL import Image, ImageDraw, ImageFont
import os, platform, json
from typing import List, Dict, Tuple, Any

# ---------------------------
# Demo rules (your mapping)
# ---------------------------
RULES: Dict[str, Dict[str, str]] = {
    "Uttar Pradesh": {
        "V_STATE_RESTRICTION_EN": "FOR SALE IN UTTAR PRADESH ONLY",
        "V_STATE_RESTRICTION_REG": "उप में केवल बिक्री के लिए",
        "V_AGE_RESTR_EN": "Not for sale to persons below 25 years of age.",
        "NOT_SOLD_IN": "Not for sale in Bihar",
        "V_HEALTH_WARN_EN": "CONSUMPTION OF ALCOHOL IS INJURIOUS TO HEALTH,\n BE SAFE-DON'T DRINK AND DRIVE.",
        "V_HEALTH_WARN_REG": "शराब स्वास्थ्य के लिए हानिकारक है.",
        "IMPORT_LISC": "1001245678",
        "Lisc_no": "1234567890",
        "V_StateLicense": "UP. EXCISE REGD. NO. E AC-45/110W-106046"
    },
    "Rajasthan": {
        "V_STATE_RESTRICTION_EN": "FOR SALE IN Rajasthan ONLY",
        "V_STATE_RESTRICTION_REG": "उप में केवल बिक्री के लिए",
        "V_AGE_RESTR_EN": "Not for sale to persons below 21 years of age.",
        "NOT_SOLD_IN": "Not for sale in Madhya Pradesh",
        "V_HEALTH_WARN_EN": "CONSUMPTION OF ALCOHOL IS INJURIOUS TO HEALTH,\n BE SAFE-DON'T DRINK AND DRIVE.",
        "V_HEALTH_WARN_REG": "शराब स्वास्थ्य के लिए हानिकारक है.",
        "IMPORT_LISC": "1001234567",
        "V_StateLicense": "RA. EXCISE REGD. NO. E AC-45/110W-10000"
    }
}

# ---------------------------
# Script detectors
# ---------------------------
def has_devanagari(text: str) -> bool:
    return any('\u0900' <= ch <= '\u097F' for ch in text)

def has_bengali(text: str) -> bool:
    return any('\u0980' <= ch <= '\u09FF' for ch in text)

# ---------------------------
# Candidate font lists (exhaustive common paths & names)
# ---------------------------
def candidate_fonts_for_script(script: str) -> List[str]:
    system = platform.system()
    cands = []
    if script == "devanagari":
        # common Devanagari fonts & likely install locations
        cands = [
            "NotoSansDevanagari-Regular.ttf",
            "NotoSansDevanagari-Regular",
            "Lohit-Devanagari.ttf",
            "Mukta-Regular.ttf",
            "Devanagari Sangam MN.ttf",
            "Mangal.ttf",                # Windows
            "Arial Unicode.ttf",
            "Arial Unicode MS.ttf",
            "/usr/share/fonts/truetype/noto/NotoSansDevanagari-Regular.ttf",
            "/usr/share/fonts/truetype/lohit/lohit-devanagari.ttf",
            "/usr/share/fonts/truetype/mukta/Mukta-Regular.ttf",
            "/Library/Fonts/Mukta-Regular.ttf",
            "/Library/Fonts/Devanagari Sangam MN.ttf",
            "/System/Library/Fonts/Supplemental/Devanagari Sangam MN.ttf",
            "DejaVuSans.ttf"
        ]
    elif script == "bengali":
        cands = [
            "NotoSansBengali-Regular.ttf",
            "NotoSansBengali-Regular",
            "Lohit-Bengali.ttf",
            "Bangla.ttf",
            "/usr/share/fonts/truetype/noto/NotoSansBengali-Regular.ttf",
            "/usr/share/fonts/truetype/lohit/lohit-bengali.ttf",
            "DejaVuSans.ttf"
        ]
    else:
        # latin / fallback
        cands = [
            "DejaVuSans.ttf",
            "Arial.ttf",
            "LiberationSans-Regular.ttf",
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
        ]

    # Also include a few OS-specific absolute paths that are common
    if system == "Darwin":
        cands += [
            "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
            "/System/Library/Fonts/Supplemental/Arial Unicode MS.ttf",
            "/Library/Fonts/Arial Unicode.ttf"
        ]
    elif system == "Windows":
        cands += [
            "C:/Windows/Fonts/arialuni.ttf",
            "C:/Windows/Fonts/arial.ttf",
            "C:/Windows/Fonts/segoeui.ttf",
            "C:/Windows/Fonts/Mangal.ttf"
        ]
    else:
        cands += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            "/usr/share/fonts/truetype/freefont/FreeSans.ttf"
        ]
    # return unique preserving order
    seen = set()
    out = []
    for f in cands:
        if f not in seen:
            seen.add(f)
            out.append(f)
    return out

# ---------------------------
# Try to instantiate a font given a candidate (tries bare name then absolute path)
# ---------------------------
def try_load_font(candidate: str, size: int):
    # try absolute path first
    try:
        if os.path.exists(candidate):
            return ImageFont.truetype(candidate, size)
    except Exception:
        pass
    # then try by file name (if installed in system font directories)
    try:
        return ImageFont.truetype(candidate, size)
    except Exception:
        pass
    return None

# ---------------------------
# Evaluate if a font "looks valid" for given text
# We'll use a heuristic: render the text to bbox and check width>0.
# (Black squares often still produce width; we cannot perfectly detect tofu, but we prioritize Indic fonts.)
# ---------------------------
def font_supports_text(font: ImageFont.FreeTypeFont, text: str) -> bool:
    try:
        # render bbox for sample text
        dummy = Image.new("L", (10,10))
        d = ImageDraw.Draw(dummy)
        bbox = d.textbbox((0,0), text, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        return (w > 0 and h > 0)
    except Exception:
        return False

# ---------------------------
# Best font & size finder:
#   - pick top candidate fonts for script
#   - for each, binary-search largest size that fits (both width & height)
#   - pick the font+size that gives the highest coverage ratio (min(width_ratio, height_ratio))
# ---------------------------
def find_best_font_for_box(draw: ImageDraw.ImageDraw, text: str, box_w: int, box_h: int,
                           max_size: int = 400, min_size: int = 6) -> ImageFont.FreeTypeFont:
    # choose script
    if has_devanagari(text):
        candidates = candidate_fonts_for_script("devanagari")
    elif has_bengali(text):
        candidates = candidate_fonts_for_script("bengali")
    else:
        candidates = candidate_fonts_for_script("latin")

    best_font = None
    best_score = -1.0  # coverage ratio

    for cand in candidates:
        # quick try to load at a medium size to check support
        test_font = try_load_font(cand, 40)
        if test_font is None:
            continue
        if not font_supports_text(test_font, text):
            # skip fonts that produce no measurable glyph area
            continue

        # binary search for largest size that fits
        lo = min_size
        hi = max_size
        chosen_size = min_size
        while lo <= hi:
            mid = (lo + hi) // 2
            f_mid = try_load_font(cand, mid)
            if f_mid is None:
                hi = mid - 1
                continue
            bbox = draw.textbbox((0,0), text, font=f_mid)
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= box_w and h <= box_h:
                chosen_size = mid
                lo = mid + 1
            else:
                hi = mid - 1

        # evaluate coverage ratio at chosen_size
        f_final = try_load_font(cand, chosen_size)
        if f_final is None:
            continue
        bboxf = draw.textbbox((0,0), text, font=f_final)
        w_f = bboxf[2] - bboxf[0]
        h_f = bboxf[3] - bboxf[1]

        width_ratio = w_f / box_w if box_w>0 else 0
        height_ratio = h_f / box_h if box_h>0 else 0
        score = min(width_ratio, height_ratio)  # prefer balanced fill

        # prefer fonts that fill more (closer to 1), tie-breaker larger area
        if score > best_score or (abs(score - best_score) < 1e-6 and (w_f*h_f) > (best_font and (draw.textbbox((0,0), text, font=best_font)[2]-draw.textbbox((0,0), text, font=best_font)[0])*(draw.textbbox((0,0), text, font=best_font)[3]-draw.textbbox((0,0), text, font=best_font)[1]) if best_font else 0)):
            best_score = score
            best_font = f_final

        # if score is almost perfect, we can stop early
        if best_score >= 0.98:
            break

    # fallback: system default at a reasonable size
    if best_font is None:
        # try default system fonts quickly
        for s in (40, 20, 12):
            try:
                f = ImageFont.truetype("DejaVuSans.ttf", s)
                if font_supports_text(f, text):
                    return f
            except:
                try:
                    f = ImageFont.load_default()
                    return f
                except:
                    pass
        return ImageFont.load_default()

    return best_font

# ---------------------------
# percent => pixels helper
# ---------------------------
def percent_to_pixels(bbox_pct: Tuple[float,float,float,float], image_size: Tuple[int,int]) -> Tuple[int,int,int,int]:
    W,H = image_size
    x_pct, y_pct, w_pct, h_pct = bbox_pct
    left = int((x_pct/100.0) * W)
    top  = int((y_pct/100.0) * H)
    w_px = int((w_pct/100.0) * W)
    h_px = int((h_pct/100.0) * H)
    return left, top, w_px, h_px

# ---------------------------
# Main renderer (final)
# ---------------------------
def render_label(master_img_path: str,
                 template_entry: Dict[str,Any],
                 state_name: str,
                 rules: Dict[str,Dict[str,str]],
                 output_path: str,
                 debug: bool = True) -> str:

    if not os.path.isfile(master_img_path):
        raise FileNotFoundError(f"Master image not found: {master_img_path}")
    img = Image.open(master_img_path).convert("RGBA")
    W,H = img.size
    draw = ImageDraw.Draw(img)

    regions = template_entry.get("regions", [])
    state_rules = rules.get(state_name)
    if state_rules is None:
        raise ValueError(f"No rules for state: {state_name}")

    for region in regions:
        label = region.get("label")
        if label is None:
            continue
        if label not in state_rules:
            if debug:
                print(f"Skipping {label}: no rule for {state_name}")
            continue

        text = state_rules[label]

        left, top, w_px, h_px = percent_to_pixels((region.get("x",0), region.get("y",0), region.get("width",0), region.get("height",0)), (W,H))
        w_px = max(1, w_px)
        h_px = max(1, h_px)

        # find best font that maximizes fill
        best_font = find_best_font_for_box(draw, text, w_px, h_px)

        # measure final bbox
        tb = draw.textbbox((0,0), text, font=best_font)
        text_w = tb[2] - tb[0]
        text_h = tb[3] - tb[1]

        # LEFT aligned horizontally with small padding
        padding_x = max(3, int(0.03 * w_px))
        x_text = left + padding_x

        # TRUE vertical center (account for font bbox baseline)
        y_text = top + (h_px - text_h) / 2 - tb[1]

        # clamp to avoid overflow
        if y_text < top:
            y_text = top
        if y_text + text_h > top + h_px:
            y_text = top + h_px - text_h

        # Draw text directly on label (no white box)
        draw.text((x_text, y_text), text, fill=(0,0,0,255), font=best_font)

        # debug: bounding box only
        if debug:
            draw.rectangle([left, top, left + w_px, top + h_px], outline=(255,0,0,200), width=1)

    # save
    out_dir = os.path.dirname(output_path) or "."
    os.makedirs(out_dir, exist_ok=True)
    img.convert("RGB").save(output_path, format="PNG")
    print("✅ Final label saved to:", output_path)
    return output_path

# ---------------------------
# ========== USER CONFIG ==========
# Edit only these paths if needed
MASTER_IMAGE_PATH = r"/Users/ayushtiwari/Desktop/label/Master Label Sample.png"
TEMPLATE_JSON_PATH = r"/Users/ayushtiwari/Desktop/label/template_clean.json"
OUTPUT_PATH = r"/Users/ayushtiwari/Desktop/label/output/final_label_v3.png"
STATE = "Uttar Pradesh"
# ---------------------------------

# Load template (try to be flexible)
def load_template_json(path: str) -> List[Dict[str,Any]]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

templates = load_template_json(TEMPLATE_JSON_PATH)
template_entry = templates[0] if len(templates)>0 else None
if template_entry is None:
    raise ValueError("No templates found in JSON")

# Run
render_label(MASTER_IMAGE_PATH, template_entry, STATE, RULES, OUTPUT_PATH, debug=True)


✅ Final label saved to: /Users/ayushtiwari/Desktop/label/output/final_label_v3.png


'/Users/ayushtiwari/Desktop/label/output/final_label_v3.png'

In [64]:
# FINAL - Robust rendering cell (copy-paste entire cell)
from PIL import Image, ImageDraw, ImageFont
import os, platform, json
from typing import List, Dict, Tuple, Any

# ---------------------------
# Demo rules (your mapping)
# ---------------------------
RULES: Dict[str, Dict[str, str]] = {
    "Uttar Pradesh": {
        "V_STATE_RESTRICTION_EN": "FOR SALE IN U.P. ONLY",
        "V_STATE_RESTRICTION_REG": "उप में केवल बिक्री के लिए",
        "V_AGE_RESTR_EN": "Not for sale to persons below 25 years of age.",
        "NOT_SOLD_IN": "Not for sale in Bihar",
        "V_HEALTH_WARN_EN": "CONSUMPTION OF ALCOHOL IS INJURIOUS TO HEALTH,\n BE SAFE-DON'T DRINK AND DRIVE.",
        "V_HEALTH_WARN_REG": "शराब स्वास्थ्य के लिए हानिकारक है.",
        "IMPORT_LISC": "1001245678",
        "Lisc_no": "1234567890",
        "V_StateLicense": "UP. EXCISE REGD. NO. E AC-45/110W-106046"
    },
    "Rajasthan": {
        "V_STATE_RESTRICTION_EN": "FOR SALE IN RAJASTHAN ONLY",
        "V_STATE_RESTRICTION_REG": "उप में केवल बिक्री के लिए",
        "V_AGE_RESTR_EN": "Not for sale to persons below 21 years of age.",
        "NOT_SOLD_IN": "Not for sale in Madhya Pradesh",
        "V_HEALTH_WARN_EN": "CONSUMPTION OF ALCOHOL IS INJURIOUS TO HEALTH,\n BE SAFE-DON'T DRINK AND DRIVE.",
        "V_HEALTH_WARN_REG": "शराब स्वास्थ्य के लिए हानिकारक है.",
        "IMPORT_LISC": "1001234567",
        "Lisc_no": "1234567890",

        "V_StateLicense": "RA. EXCISE REGD. NO. E AC-45/110W-10000"
    }
}

# ---------------------------
# Script detectors
# ---------------------------
def has_devanagari(text: str) -> bool:
    return any('\u0900' <= ch <= '\u097F' for ch in text)

def has_bengali(text: str) -> bool:
    return any('\u0980' <= ch <= '\u09FF' for ch in text)

# ---------------------------
# Candidate font lists (exhaustive common paths & names)
# ---------------------------
def candidate_fonts_for_script(script: str) -> List[str]:
    system = platform.system()
    cands = []
    if script == "devanagari":
        # common Devanagari fonts & likely install locations
        cands = [
            "NotoSansDevanagari-Regular.ttf",
            "NotoSansDevanagari-Regular",
            "Lohit-Devanagari.ttf",
            "Mukta-Regular.ttf",
            "Devanagari Sangam MN.ttf",
            "Mangal.ttf",                # Windows
            "Arial Unicode.ttf",
            "Arial Unicode MS.ttf",
            "/usr/share/fonts/truetype/noto/NotoSansDevanagari-Regular.ttf",
            "/usr/share/fonts/truetype/lohit/lohit-devanagari.ttf",
            "/usr/share/fonts/truetype/mukta/Mukta-Regular.ttf",
            "/Library/Fonts/Mukta-Regular.ttf",
            "/Library/Fonts/Devanagari Sangam MN.ttf",
            "/System/Library/Fonts/Supplemental/Devanagari Sangam MN.ttf",
            "DejaVuSans.ttf"
        ]
    elif script == "bengali":
        cands = [
            "NotoSansBengali-Regular.ttf",
            "NotoSansBengali-Regular",
            "Lohit-Bengali.ttf",
            "Bangla.ttf",
            "/usr/share/fonts/truetype/noto/NotoSansBengali-Regular.ttf",
            "/usr/share/fonts/truetype/lohit/lohit-bengali.ttf",
            "DejaVuSans.ttf"
        ]
    else:
        # latin / fallback
        cands = [
            "DejaVuSans.ttf",
            "Arial.ttf",
            "LiberationSans-Regular.ttf",
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
        ]

    # Also include a few OS-specific absolute paths that are common
    if system == "Darwin":
        cands += [
            "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
            "/System/Library/Fonts/Supplemental/Arial Unicode MS.ttf",
            "/Library/Fonts/Arial Unicode.ttf"
        ]
    elif system == "Windows":
        cands += [
            "C:/Windows/Fonts/arialuni.ttf",
            "C:/Windows/Fonts/arial.ttf",
            "C:/Windows/Fonts/segoeui.ttf",
            "C:/Windows/Fonts/Mangal.ttf"
        ]
    else:
        cands += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            "/usr/share/fonts/truetype/freefont/FreeSans.ttf"
        ]
    # return unique preserving order
    seen = set()
    out = []
    for f in cands:
        if f not in seen:
            seen.add(f)
            out.append(f)
    return out

# ---------------------------
# Try to instantiate a font given a candidate (tries bare name then absolute path)
# ---------------------------
def try_load_font(candidate: str, size: int):
    # try absolute path first
    try:
        if os.path.exists(candidate):
            return ImageFont.truetype(candidate, size)
    except Exception:
        pass
    # then try by file name (if installed in system font directories)
    try:
        return ImageFont.truetype(candidate, size)
    except Exception:
        pass
    return None

# ---------------------------
# Evaluate if a font "looks valid" for given text
# We'll use a heuristic: render the text to bbox and check width>0.
# (Black squares often still produce width; we cannot perfectly detect tofu, but we prioritize Indic fonts.)
# ---------------------------
def font_supports_text(font: ImageFont.FreeTypeFont, text: str) -> bool:
    try:
        # render bbox for sample text
        dummy = Image.new("L", (10,10))
        d = ImageDraw.Draw(dummy)
        bbox = d.textbbox((0,0), text, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        return (w > 0 and h > 0)
    except Exception:
        return False

# ---------------------------
# Best font & size finder:
#   - pick top candidate fonts for script
#   - for each, binary-search largest size that fits (both width & height)
#   - pick the font+size that gives the highest coverage ratio (min(width_ratio, height_ratio))
# ---------------------------
def find_best_font_for_box(draw: ImageDraw.ImageDraw, text: str, box_w: int, box_h: int,
                           max_size: int = 400, min_size: int = 6) -> ImageFont.FreeTypeFont:
    # choose script
    if has_devanagari(text):
        candidates = candidate_fonts_for_script("devanagari")
    elif has_bengali(text):
        candidates = candidate_fonts_for_script("bengali")
    else:
        candidates = candidate_fonts_for_script("latin")

    best_font = None
    best_score = -1.0  # coverage ratio

    for cand in candidates:
        # quick try to load at a medium size to check support
        test_font = try_load_font(cand, 40)
        if test_font is None:
            continue
        if not font_supports_text(test_font, text):
            # skip fonts that produce no measurable glyph area
            continue

        # binary search for largest size that fits
        lo = min_size
        hi = max_size
        chosen_size = min_size
        while lo <= hi:
            mid = (lo + hi) // 2
            f_mid = try_load_font(cand, mid)
            if f_mid is None:
                hi = mid - 1
                continue
            bbox = draw.textbbox((0,0), text, font=f_mid)
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= box_w and h <= box_h:
                chosen_size = mid
                lo = mid + 1
            else:
                hi = mid - 1

        # evaluate coverage ratio at chosen_size
        f_final = try_load_font(cand, chosen_size)
        if f_final is None:
            continue
        bboxf = draw.textbbox((0,0), text, font=f_final)
        w_f = bboxf[2] - bboxf[0]
        h_f = bboxf[3] - bboxf[1]

        width_ratio = w_f / box_w if box_w>0 else 0
        height_ratio = h_f / box_h if box_h>0 else 0
        score = min(width_ratio, height_ratio)  # prefer balanced fill

        # prefer fonts that fill more (closer to 1), tie-breaker larger area
        if score > best_score or (abs(score - best_score) < 1e-6 and (w_f*h_f) > (best_font and (draw.textbbox((0,0), text, font=best_font)[2]-draw.textbbox((0,0), text, font=best_font)[0])*(draw.textbbox((0,0), text, font=best_font)[3]-draw.textbbox((0,0), text, font=best_font)[1]) if best_font else 0)):
            best_score = score
            best_font = f_final

        # if score is almost perfect, we can stop early
        if best_score >= 0.98:
            break

    # fallback: system default at a reasonable size
    if best_font is None:
        # try default system fonts quickly
        for s in (40, 20, 12):
            try:
                f = ImageFont.truetype("DejaVuSans.ttf", s)
                if font_supports_text(f, text):
                    return f
            except:
                try:
                    f = ImageFont.load_default()
                    return f
                except:
                    pass
        return ImageFont.load_default()

    return best_font

# ---------------------------
# percent => pixels helper
# ---------------------------
def percent_to_pixels(bbox_pct: Tuple[float,float,float,float], image_size: Tuple[int,int]) -> Tuple[int,int,int,int]:
    W,H = image_size
    x_pct, y_pct, w_pct, h_pct = bbox_pct
    left = int((x_pct/100.0) * W)
    top  = int((y_pct/100.0) * H)
    w_px = int((w_pct/100.0) * W)
    h_px = int((h_pct/100.0) * H)
    return left, top, w_px, h_px

# ---------------------------
# Main renderer (final)
# ---------------------------
def render_label(master_img_path: str,
                 template_entry: Dict[str,Any],
                 state_name: str,
                 rules: Dict[str,Dict[str,str]],
                 output_path: str,
                 debug: bool = True) -> str:

    if not os.path.isfile(master_img_path):
        raise FileNotFoundError(f"Master image not found: {master_img_path}")
    img = Image.open(master_img_path).convert("RGBA")
    W,H = img.size
    draw = ImageDraw.Draw(img)

    regions = template_entry.get("regions", [])
    state_rules = rules.get(state_name)
    if state_rules is None:
        raise ValueError(f"No rules for state: {state_name}")

    for region in regions:
        label = region.get("label")
        if label is None:
            continue
        if label not in state_rules:
            if debug:
                print(f"Skipping {label}: no rule for {state_name}")
            continue

        text = state_rules[label]

        left, top, w_px, h_px = percent_to_pixels((region.get("x",0), region.get("y",0), region.get("width",0), region.get("height",0)), (W,H))
        w_px = max(1, w_px)
        h_px = max(1, h_px)

        # find best font that maximizes fill
        best_font = find_best_font_for_box(draw, text, w_px, h_px)

        # measure final bbox
        tb = draw.textbbox((0,0), text, font=best_font)
        text_w = tb[2] - tb[0]
        text_h = tb[3] - tb[1]

        # LEFT aligned horizontally with small padding
        padding_x = max(3, int(0.03 * w_px))
        x_text = left + padding_x

        # TRUE vertical center (account for font bbox baseline)
        y_text = top + (h_px - text_h) / 2 - tb[1]

        # clamp to avoid overflow
        if y_text < top:
            y_text = top
        if y_text + text_h > top + h_px:
            y_text = top + h_px - text_h

        # Draw text directly on label (no white box)
        draw.text((x_text, y_text), text, fill=(0,0,0,255), font=best_font)

        # debug: bounding box only
        if debug:
            draw.rectangle([left, top, left + w_px, top + h_px], outline=(255,0,0,200), width=1)

    # save
    out_dir = os.path.dirname(output_path) or "."
    os.makedirs(out_dir, exist_ok=True)
    img.convert("RGB").save(output_path, format="PNG")
    print("✅ Final label saved to:", output_path)
    return output_path

# ---------------------------
# ========== USER CONFIG ==========
# Edit only these paths if needed
MASTER_IMAGE_PATH = r"/Users/ayushtiwari/Desktop/label/Master Label Sample.png"
TEMPLATE_JSON_PATH = r"/Users/ayushtiwari/Desktop/label/template_clean.json"
OUTPUT_PATH = r"/Users/ayushtiwari/Desktop/label/output/final_label_v3_R.png"
STATE = "Rajasthan"
# ---------------------------------

# Load template (try to be flexible)
def load_template_json(path: str) -> List[Dict[str,Any]]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

templates = load_template_json(TEMPLATE_JSON_PATH)
template_entry = templates[0] if len(templates)>0 else None
if template_entry is None:
    raise ValueError("No templates found in JSON")

# Run
render_label(MASTER_IMAGE_PATH, template_entry, STATE, RULES, OUTPUT_PATH, debug=True)


✅ Final label saved to: /Users/ayushtiwari/Desktop/label/output/final_label_v3_R.png


'/Users/ayushtiwari/Desktop/label/output/final_label_v3_R.png'

In [63]:
# FINAL - Robust rendering cell (copy-paste entire cell)
from PIL import Image, ImageDraw, ImageFont
import os, platform, json
from typing import List, Dict, Tuple, Any

# ---------------------------
# Demo rules (your mapping)
# ---------------------------
RULES: Dict[str, Dict[str, str]] = {
    "Uttar Pradesh": {
        "V_STATE_RESTRICTION_EN": "FOR SALE IN U.P. ONLY",
        "V_STATE_RESTRICTION_REG": "उप में केवल बिक्री के लिए",
        "V_AGE_RESTR_EN": "Not for sale to persons below 25 years of age.",
        "NOT_SOLD_IN": "Not for sale in Bihar",
        "V_HEALTH_WARN_EN": "CONSUMPTION OF ALCOHOL IS INJURIOUS TO HEALTH,\n BE SAFE-DON'T DRINK AND DRIVE.",
        "V_HEALTH_WARN_REG": "शराब स्वास्थ्य के लिए हानिकारक है.",
        "IMPORT_LISC": "1001245678",
        "Lisc_no": "1234567890",
        "V_StateLicense": "UP. EXCISE REGD. NO. E AC-45/110W-106046"
    },
    "Rajasthan": {
        "V_STATE_RESTRICTION_EN": "FOR SALE IN RAJASTHAN ONLY",
        "V_STATE_RESTRICTION_REG": "उप में केवल बिक्री के लिए",
        "V_AGE_RESTR_EN": "Not for sale to persons below 21 years of age.",
        "NOT_SOLD_IN": "Not for sale in Madhya Pradesh",
        "V_HEALTH_WARN_EN": "CONSUMPTION OF ALCOHOL IS INJURIOUS TO HEALTH,\n BE SAFE-DON'T DRINK AND DRIVE.",
        "V_HEALTH_WARN_REG": "शराब स्वास्थ्य के लिए हानिकारक है.",
        "IMPORT_LISC": "1001234567",
        "Lisc_no": "1234567890",

        "V_StateLicense": "RA. EXCISE REGD. NO. E AC-45/110W-10000"
    }
}

# ---------------------------
# Script detectors
# ---------------------------
def has_devanagari(text: str) -> bool:
    return any('\u0900' <= ch <= '\u097F' for ch in text)

def has_bengali(text: str) -> bool:
    return any('\u0980' <= ch <= '\u09FF' for ch in text)

# ---------------------------
# Candidate font lists (exhaustive common paths & names)
# ---------------------------
def candidate_fonts_for_script(script: str) -> List[str]:
    system = platform.system()
    cands = []
    if script == "devanagari":
        # common Devanagari fonts & likely install locations
        cands = [
            "NotoSansDevanagari-Regular.ttf",
            "NotoSansDevanagari-Regular",
            "Lohit-Devanagari.ttf",
            "Mukta-Regular.ttf",
            "Devanagari Sangam MN.ttf",
            "Mangal.ttf",                # Windows
            "Arial Unicode.ttf",
            "Arial Unicode MS.ttf",
            "/usr/share/fonts/truetype/noto/NotoSansDevanagari-Regular.ttf",
            "/usr/share/fonts/truetype/lohit/lohit-devanagari.ttf",
            "/usr/share/fonts/truetype/mukta/Mukta-Regular.ttf",
            "/Library/Fonts/Mukta-Regular.ttf",
            "/Library/Fonts/Devanagari Sangam MN.ttf",
            "/System/Library/Fonts/Supplemental/Devanagari Sangam MN.ttf",
            "DejaVuSans.ttf"
        ]
    elif script == "bengali":
        cands = [
            "NotoSansBengali-Regular.ttf",
            "NotoSansBengali-Regular",
            "Lohit-Bengali.ttf",
            "Bangla.ttf",
            "/usr/share/fonts/truetype/noto/NotoSansBengali-Regular.ttf",
            "/usr/share/fonts/truetype/lohit/lohit-bengali.ttf",
            "DejaVuSans.ttf"
        ]
    else:
        # latin / fallback
        cands = [
            "DejaVuSans.ttf",
            "Arial.ttf",
            "LiberationSans-Regular.ttf",
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
        ]

    # Also include a few OS-specific absolute paths that are common
    if system == "Darwin":
        cands += [
            "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
            "/System/Library/Fonts/Supplemental/Arial Unicode MS.ttf",
            "/Library/Fonts/Arial Unicode.ttf"
        ]
    elif system == "Windows":
        cands += [
            "C:/Windows/Fonts/arialuni.ttf",
            "C:/Windows/Fonts/arial.ttf",
            "C:/Windows/Fonts/segoeui.ttf",
            "C:/Windows/Fonts/Mangal.ttf"
        ]
    else:
        cands += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            "/usr/share/fonts/truetype/freefont/FreeSans.ttf"
        ]
    # return unique preserving order
    seen = set()
    out = []
    for f in cands:
        if f not in seen:
            seen.add(f)
            out.append(f)
    return out

# ---------------------------
# Try to instantiate a font given a candidate (tries bare name then absolute path)
# ---------------------------
def try_load_font(candidate: str, size: int):
    # try absolute path first
    try:
        if os.path.exists(candidate):
            return ImageFont.truetype(candidate, size)
    except Exception:
        pass
    # then try by file name (if installed in system font directories)
    try:
        return ImageFont.truetype(candidate, size)
    except Exception:
        pass
    return None

# ---------------------------
# Evaluate if a font "looks valid" for given text
# We'll use a heuristic: render the text to bbox and check width>0.
# (Black squares often still produce width; we cannot perfectly detect tofu, but we prioritize Indic fonts.)
# ---------------------------
def font_supports_text(font: ImageFont.FreeTypeFont, text: str) -> bool:
    try:
        # render bbox for sample text
        dummy = Image.new("L", (10,10))
        d = ImageDraw.Draw(dummy)
        bbox = d.textbbox((0,0), text, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        return (w > 0 and h > 0)
    except Exception:
        return False

# ---------------------------
# Best font & size finder:
#   - pick top candidate fonts for script
#   - for each, binary-search largest size that fits (both width & height)
#   - pick the font+size that gives the highest coverage ratio (min(width_ratio, height_ratio))
# ---------------------------
def find_best_font_for_box(draw: ImageDraw.ImageDraw, text: str, box_w: int, box_h: int,
                           max_size: int = 400, min_size: int = 6) -> ImageFont.FreeTypeFont:
    # choose script
    if has_devanagari(text):
        candidates = candidate_fonts_for_script("devanagari")
    elif has_bengali(text):
        candidates = candidate_fonts_for_script("bengali")
    else:
        candidates = candidate_fonts_for_script("latin")

    best_font = None
    best_score = -1.0  # coverage ratio

    for cand in candidates:
        # quick try to load at a medium size to check support
        test_font = try_load_font(cand, 40)
        if test_font is None:
            continue
        if not font_supports_text(test_font, text):
            # skip fonts that produce no measurable glyph area
            continue

        # binary search for largest size that fits
        lo = min_size
        hi = max_size
        chosen_size = min_size
        while lo <= hi:
            mid = (lo + hi) // 2
            f_mid = try_load_font(cand, mid)
            if f_mid is None:
                hi = mid - 1
                continue
            bbox = draw.textbbox((0,0), text, font=f_mid)
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            if w <= box_w and h <= box_h:
                chosen_size = mid
                lo = mid + 1
            else:
                hi = mid - 1

        # evaluate coverage ratio at chosen_size
        f_final = try_load_font(cand, chosen_size)
        if f_final is None:
            continue
        bboxf = draw.textbbox((0,0), text, font=f_final)
        w_f = bboxf[2] - bboxf[0]
        h_f = bboxf[3] - bboxf[1]

        width_ratio = w_f / box_w if box_w>0 else 0
        height_ratio = h_f / box_h if box_h>0 else 0
        score = min(width_ratio, height_ratio)  # prefer balanced fill

        # prefer fonts that fill more (closer to 1), tie-breaker larger area
        if score > best_score or (abs(score - best_score) < 1e-6 and (w_f*h_f) > (best_font and (draw.textbbox((0,0), text, font=best_font)[2]-draw.textbbox((0,0), text, font=best_font)[0])*(draw.textbbox((0,0), text, font=best_font)[3]-draw.textbbox((0,0), text, font=best_font)[1]) if best_font else 0)):
            best_score = score
            best_font = f_final

        # if score is almost perfect, we can stop early
        if best_score >= 0.98:
            break

    # fallback: system default at a reasonable size
    if best_font is None:
        # try default system fonts quickly
        for s in (40, 20, 12):
            try:
                f = ImageFont.truetype("DejaVuSans.ttf", s)
                if font_supports_text(f, text):
                    return f
            except:
                try:
                    f = ImageFont.load_default()
                    return f
                except:
                    pass
        return ImageFont.load_default()

    return best_font

# ---------------------------
# percent => pixels helper
# ---------------------------
def percent_to_pixels(bbox_pct: Tuple[float,float,float,float], image_size: Tuple[int,int]) -> Tuple[int,int,int,int]:
    W,H = image_size
    x_pct, y_pct, w_pct, h_pct = bbox_pct
    left = int((x_pct/100.0) * W)
    top  = int((y_pct/100.0) * H)
    w_px = int((w_pct/100.0) * W)
    h_px = int((h_pct/100.0) * H)
    return left, top, w_px, h_px

# ---------------------------
# Main renderer (final)
# ---------------------------
def render_label(master_img_path: str,
                 template_entry: Dict[str,Any],
                 state_name: str,
                 rules: Dict[str,Dict[str,str]],
                 output_path: str,
                 debug: bool = True) -> str:

    if not os.path.isfile(master_img_path):
        raise FileNotFoundError(f"Master image not found: {master_img_path}")
    img = Image.open(master_img_path).convert("RGBA")
    W,H = img.size
    draw = ImageDraw.Draw(img)

    regions = template_entry.get("regions", [])
    state_rules = rules.get(state_name)
    if state_rules is None:
        raise ValueError(f"No rules for state: {state_name}")

    for region in regions:
        label = region.get("label")
        if label is None:
            continue
        if label not in state_rules:
            if debug:
                print(f"Skipping {label}: no rule for {state_name}")
            continue

        text = state_rules[label]

        left, top, w_px, h_px = percent_to_pixels((region.get("x",0), region.get("y",0), region.get("width",0), region.get("height",0)), (W,H))
        w_px = max(1, w_px)
        h_px = max(1, h_px)

        # find best font that maximizes fill
        best_font = find_best_font_for_box(draw, text, w_px, h_px)

        # measure final bbox
        tb = draw.textbbox((0,0), text, font=best_font)
        text_w = tb[2] - tb[0]
        text_h = tb[3] - tb[1]

        # LEFT aligned horizontally with small padding
        padding_x = max(3, int(0.03 * w_px))
        x_text = left + padding_x

        # TRUE vertical center (account for font bbox baseline)
        y_text = top + (h_px - text_h) / 2 - tb[1]

        # clamp to avoid overflow
        if y_text < top:
            y_text = top
        if y_text + text_h > top + h_px:
            y_text = top + h_px - text_h

        # Draw text directly on label (no white box)
        draw.text((x_text, y_text), text, fill=(0,0,0,255), font=best_font)

        # debug: bounding box only
        if debug:
            draw.rectangle([left, top, left + w_px, top + h_px], outline=(255,0,0,200), width=1)

    # save
    out_dir = os.path.dirname(output_path) or "."
    os.makedirs(out_dir, exist_ok=True)
    img.convert("RGB").save(output_path, format="PNG")
    print("✅ Final label saved to:", output_path)
    return output_path

# ---------------------------
# ========== USER CONFIG ==========
# Edit only these paths if needed
MASTER_IMAGE_PATH = r"/Users/ayushtiwari/Desktop/label/Master Label Sample.png"
TEMPLATE_JSON_PATH = r"/Users/ayushtiwari/Desktop/label/template_clean.json"
OUTPUT_PATH = r"/Users/ayushtiwari/Desktop/label/output/final_label_v3_R_position.png"
STATE = "Rajasthan"
# ---------------------------------

# Load template (try to be flexible)
def load_template_json(path: str) -> List[Dict[str,Any]]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

templates = load_template_json(TEMPLATE_JSON_PATH)
TARGET_IMAGE = "jw_position.png"

template_entry = next(
    t for t in templates 
    if t["image"] == TARGET_IMAGE
)
if template_entry is None:
    raise ValueError("No templates found in JSON")

# Run
render_label(MASTER_IMAGE_PATH, template_entry, STATE, RULES, OUTPUT_PATH, debug=True)


✅ Final label saved to: /Users/ayushtiwari/Desktop/label/output/final_label_v3_R_position.png


'/Users/ayushtiwari/Desktop/label/output/final_label_v3_R_position.png'